In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from konlpy.tag import Mecab
from copy import deepcopy
tagger = Mecab()
flatten = lambda l: [item for sublist in l for item in sublist]
from data_utils import *

In [3]:
USE_CUDA = torch.cuda.is_available()

In [4]:
data = open("contextual.iob","r",encoding="utf-8").readlines()

In [5]:
train_data=[]
history=[["<null>"]]
for d in data:
    if d=="\n":
        history=[["<null>"]]
        continue
    dd = d.replace("\n","").split("\t")
    if dd[1]=="":
        bot = tagger.morphs(dd[0])
        history.append(bot)
    else:
        user = dd[0].split()
        tag = dd[1].split()
        intent = dd[2]
        temp = deepcopy(history)
        train_data.append([temp,user,tag,intent])
        history.append(user)

In [6]:
historys, currents, slots, intents = list(zip(*train_data))

In [7]:
vocab = list(set(flatten(currents)))
slot_vocab = list(set(flatten(slots)))
intent_vocab = list(set(intents))

In [8]:
word2index={"<pad>" : 0, "<unk>" : 1, "<null>" : 2, "<s>" : 3, "</s>" : 4}
for vo in vocab:
    if word2index.get(vo)==None:
        word2index[vo] = len(word2index)
        
slot2index={"<pad>" : 0}
for vo in slot_vocab:
    if slot2index.get(vo)==None:
        slot2index[vo] = len(slot2index)
        
intent2index={}
for vo in intent_vocab:
    if intent2index.get(vo)==None:
        intent2index[vo] = len(intent2index)

In [9]:
for t in train_data:
    for i,history in enumerate(t[0]):
        t[0][i] = prepare_sequence(history, word2index).view(1, -1)

    t[1] = prepare_sequence(t[1], word2index).view(1, -1)
    t[2] = prepare_sequence(t[2], slot2index).view(1, -1)
    t[3] = Variable(torch.LongTensor([intent2index[t[3]]])).view(1,-1)

In [24]:
class SDEN(nn.Module):
    def __init__(self,vocab_size,embed_size,hidden_size,slot_size,intent_size):
        super(SDEN,self).__init__()
        
        self.embed = nn.Embedding(vocab_size,embed_size)
        self.bigru_m = nn.GRU(embed_size,hidden_size,batch_first=True,bidirectional=True)
        self.bigru_c = nn.GRU(embed_size,hidden_size,batch_first=True,bidirectional=True)
        self.context_encoder = nn.Sequential(nn.Linear(hidden_size*4,hidden_size*2),
                                                               nn.Sigmoid())
        self.session_encoder = nn.GRU(hidden_size*2,hidden_size*2,batch_first=True,bidirectional=True)
        
        self.decoder_1 = nn.GRU(embed_size,hidden_size*2,batch_first=True,bidirectional=True)
        self.decoder_2 = nn.LSTM(hidden_size*4,hidden_size*2,batch_first=True,bidirectional=True)
        
        self.intent_linear = nn.Linear(hidden_size*4,intent_size)
        self.slot_linear = nn.Linear(hidden_size*4,slot_size)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self,history,current):
        batch_size = len(history)
        H= [] # encoded history
        for h in history:
            mask = h.eq(0)
            embeds = self.embed(h)
            embeds = self.dropout(embeds)
            outputs, hidden = self.bigru_m(embeds)
            real_hidden = []

            for i, o in enumerate(outputs): # B,T,D
                real_length = mask[i].data.tolist().count(0) 
                real_hidden.append(o[real_length - 1])

            H.append(torch.cat(real_hidden).view(h.size(0), -1).unsqueeze(0))
        
        M = torch.cat(H) # B,T_C,2H
        M = self.dropout(M)
        embeds = self.embed(current)
        embeds = self.dropout(embeds)
        mask = current.eq(0)
        outputs, hidden = self.bigru_c(embeds)
        real_hidden=[]
        for i, o in enumerate(outputs): # B,T,D
            real_length = mask[i].data.tolist().count(0) 
            real_hidden.append(o[real_length - 1])
        C = torch.cat(real_hidden).view(current.size(0),1, -1) # B,1,2H
        C = self.dropout(C)
        
        CONCAT = []
        for i in range(batch_size):
            m = M[i] # T_c,2H
            c = C[i] # 1,2H
            c = c.expand_as(m)
            cat = torch.cat([m,c],1)
            CONCAT.append(cat.unsqueeze(0))
        CONCAT = torch.cat(CONCAT)
        
        G = self.context_encoder(CONCAT)
        
        _,H = self.session_encoder(G) # 2,B,2H
        cell_state = torch.zeros_like(H)
        if USE_CUDA: cell_state = cell_state.cuda()
        O_1,_ = self.decoder_1(embeds)
        O_1 = self.dropout(O_1)
        
        O_2,(S_2,_) = self.decoder_2(O_1,(H,cell_state))
        O_2 = self.dropout(O_2)
        S = torch.cat([s for s in S_2],1)
        
        intent_prob = self.intent_linear(S)
        slot_prob = self.slot_linear(O_2.contiguous().view(O_2.size(0)*O_2.size(1),-1))
        
        return slot_prob, intent_prob

In [25]:
EPOCH = 20
BATCH = 32
LR = 0.001

In [31]:
EPOCH = 10

In [26]:
model = SDEN(len(word2index),100,64,len(slot2index),len(intent2index))
slot_loss_function = nn.CrossEntropyLoss(ignore_index=0)
intent_loss_function = nn.CrossEntropyLoss()
if USE_CUDA:
    model = model.cuda()
optimizer = optim.Adam(model.parameters(),lr=LR)
scheduler = optim.lr_scheduler.MultiStepLR(gamma=0.1,milestones=[EPOCH//4,EPOCH//2],optimizer=optimizer)

In [34]:
model.train()
for epoch in range(EPOCH):
    losses=[]
    scheduler.step()
    for i,batch in enumerate(data_loader(train_data,BATCH,True)):
        h,c,slot,intent = pad_to_batch(batch,word2index,slot2index)
        if USE_CUDA:
            h = [hh.cuda() for hh in h]
            c = c.cuda()
            slot = slot.cuda()
            intent = intent.cuda()
        model.zero_grad()
        slot_p, intent_p = model(h,c)

        loss_s = slot_loss_function(slot_p,slot.view(-1))
        loss_i = intent_loss_function(intent_p,intent.view(-1))
        loss = loss_s + loss_i
        losses.append(loss.data[0])
        loss.backward()
        optimizer.step()
        
        if i % 200 == 0:
            print("[%d/%d] [%d/%d] mean_loss : %.3f" % (epoch,EPOCH,i,len(train_data)//BATCH,np.mean(losses)))
            losses=[]

[0/10] [0/2422] mean_loss : 0.589
[0/10] [200/2422] mean_loss : 0.363
[0/10] [400/2422] mean_loss : 0.357
[0/10] [600/2422] mean_loss : 0.342
[0/10] [800/2422] mean_loss : 0.345
[0/10] [1000/2422] mean_loss : 0.345
[0/10] [1200/2422] mean_loss : 0.343
[0/10] [1400/2422] mean_loss : 0.348
[0/10] [1600/2422] mean_loss : 0.355
[0/10] [1800/2422] mean_loss : 0.337
[0/10] [2000/2422] mean_loss : 0.339
[0/10] [2200/2422] mean_loss : 0.352
[0/10] [2400/2422] mean_loss : 0.348
[1/10] [0/2422] mean_loss : 0.522
[1/10] [200/2422] mean_loss : 0.350
[1/10] [400/2422] mean_loss : 0.323
[1/10] [600/2422] mean_loss : 0.338
[1/10] [800/2422] mean_loss : 0.332
[1/10] [1000/2422] mean_loss : 0.343
[1/10] [1200/2422] mean_loss : 0.336
[1/10] [1400/2422] mean_loss : 0.341
[1/10] [1600/2422] mean_loss : 0.330
[1/10] [1800/2422] mean_loss : 0.355
[1/10] [2000/2422] mean_loss : 0.318
[1/10] [2200/2422] mean_loss : 0.335
[1/10] [2400/2422] mean_loss : 0.342
[2/10] [0/2422] mean_loss : 0.233
[2/10] [200/2422] 

In [35]:
import pickle

In [36]:
model = model.cpu()
torch.save(model.state_dict(),'sden.pkl')

In [37]:
pickle.dump(word2index,open('vocab.pkl','wb'))
pickle.dump(slot2index,open('slot.pkl','wb'))
pickle.dump(intent2index,open('intent.pkl','wb'))